In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("/home/apoorv/Desktop/github/activepm/data/china_data_aq.csv", index_col = 0)

/home/apoorv/Desktop/github/.env/lib/python3.6/site-packages/numpy/lib/arraysetops.py:568: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
df = df.drop(columns = ["PM10_Concentration", "NO2_Concentration", "CO_Concentration", "O3_Concentration", "SO2_Concentration", "district_id", "District Name", "city_id" , "City Name", "City-Lat"])
df = df.dropna()

In [4]:
df.head()

,station_id,time,PM25_Concentration,Station Name,latitude,longitude,City-Long,cluster_id
0,1001,2014-05-01 00:00:00,138.0,HaiDianBeiBuXinQu,40.090679,116.173553,116.407394,1
1,1001,2014-05-01 01:00:00,124.0,HaiDianBeiBuXinQu,40.090679,116.173553,116.407394,1
2,1001,2014-05-01 02:00:00,127.0,HaiDianBeiBuXinQu,40.090679,116.173553,116.407394,1
3,1001,2014-05-01 03:00:00,129.0,HaiDianBeiBuXinQu,40.090679,116.173553,116.407394,1
4,1001,2014-05-01 04:00:00,119.0,HaiDianBeiBuXinQu,40.090679,116.173553,116.407394,1


In [5]:
data = []

In [6]:
df1 = pd.read_csv("../data/beijinga_scaled.csv")
stations = df1['station_id'].unique()
stations.sort()

In [7]:
for stn in stations:
    data.append(df.groupby('station_id').get_group(stn))    

In [8]:
def daterange(start_date, end_date):
    delta = timedelta(hours=1)
    while start_date < end_date:
        yield start_date
        start_date += delta

In [9]:
times_in_df = df["time"].values

In [10]:
extra = []
for yr in range((times_in_df.shape[0])):
    if '1970' in times_in_df[yr]:
        extra.append(times_in_df[yr])
df = df[~df["time"].isin(extra)]

In [11]:
import datetime
from datetime import timedelta
start = datetime.datetime.strptime("01-05-2014 00:00:00", "%d-%m-%Y %H:%M:%S")
end = datetime.datetime.strptime("01-05-2015 01:00:00", "%d-%m-%Y %H:%M:%S")
date_generated = {start + datetime.timedelta(hours=1):x for x in range(0, (end-start).seconds)}
dates_generated = {i:x for x, i in enumerate(daterange(start, end))}
del datetime

In [12]:
from datetime import datetime
time_stamps = np.array(df['time'])
dates = np.array([datetime.strptime(dt, "%Y-%m-%d %H:%M:%S") for dt in time_stamps])

In [13]:
ts = np.array([dates_generated[i] for i in dates])

In [14]:
df["Time"] = ts
df.drop(columns = ["Station Name", "City-Long", "cluster_id", "time"], inplace = True)

In [15]:
stations_to_add = df["station_id"].values
pm_to_add = df["PM25_Concentration"].values
df.drop(columns = ["station_id"], inplace = True)
df = df - df.min()
df = df / df.max()
df['station_id'] = stations_to_add
df['PM2.5'] = pm_to_add

In [16]:
df.drop(columns = ["PM25_Concentration"], inplace = True)

In [17]:
df.head()

,latitude,longitude,Time,station_id,PM2.5
0,0.908951,0.596592,0.000000,1001,138.0
1,0.908951,0.596592,0.000114,1001,124.0
2,0.908951,0.596592,0.000228,1001,127.0
3,0.908951,0.596592,0.000343,1001,129.0
4,0.908951,0.596592,0.000457,1001,119.0


In [18]:
from sklearn.metrics import mean_squared_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ExpSineSquared
import xgboost
from pykrige.ok import OrdinaryKriging
from sklearn.model_selection import KFold

In [31]:
lastK = 7 * 24
uptillHr = 500

In [32]:
"""
Use the full dataset, but only for stations in beijijng
"""
errors = {}
stations.sort()

times = df["Time"].unique()
times.sort()

splits = 6

kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

station_split = {}

for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
    station_split[kout] = {}
    for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
        sts_test = stations[sts_test_index]
        sts_val = stations[sts_ftrain_index[sts_val_index]]
        sts_train = stations[sts_ftrain_index[sts_train_index]]
        station_split[kout][kin] = {
            'test': sts_test,
            'train': sts_val,
            'pool': sts_train,
        }
        
        
        
for i in range(6):
    for j in range(5):
        for hello, time in enumerate(times):
            if hello == uptillHr: 
                break
            try:
                hello2 = max(hello - lastK, 0)
                train_df = df[df['station_id'].isin(station_split[i][j]['train'])]
                test_df = df[df['station_id'].isin(station_split[i][j]['test'])]
                train_df = train_df[(train_df["Time"] <= time) & (train_df["Time"] > times[hello2])]
                test_df = test_df[(test_df["Time"] == time)]
                if test_df.shape[0] == 0 or train_df.shape[0] == 0:
                    continue
                X_train = train_df[["longitude", "latitude", "Time"]]
                y_train = train_df[["PM2.5"]]
                X_test = test_df[["longitude", "latitude", "Time"]]
                y_test = test_df[["PM2.5"]]
                xgb = xgboost.XGBRegressor()
                xgb.fit(X_train, y_train)
                prediction = xgb.predict(X_test)
                error = np.sqrt( mean_squared_error( prediction, y_test ))
                errors[hello] = error
            except Exception as e:
                print(e)
                continue
        break
    break

[16:48:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:54] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:48:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:49:03] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [33]:
a = np.array([[1,3], [2, 4]])

In [34]:
a

array([[1, 3],
       [2, 4]])

In [35]:
"""
Use the full dataset, but only for stations in beijijng
"""
errors2 = {}
stations.sort()

times = df["Time"].unique()
times.sort()

splits = 6

kfout = KFold(n_splits=splits, shuffle=True, random_state=0)
kfin = KFold(n_splits=splits - 1, shuffle=True, random_state=0)

station_split = {}

for kout, (sts_ftrain_index, sts_test_index) in enumerate(kfout.split(stations)):
    station_split[kout] = {}
    for kin, (sts_train_index, sts_val_index) in enumerate(kfin.split(sts_ftrain_index)):
        sts_test = stations[sts_test_index]
        sts_val = stations[sts_ftrain_index[sts_val_index]]
        sts_train = stations[sts_ftrain_index[sts_train_index]]
        station_split[kout][kin] = {
            'test': sts_test,
            'train': sts_val,
            'pool': sts_train,
        }
        
        
        
for i in range(6):
    for j in range(5):
        for hello, time in enumerate(times):
            if hello == uptillHr: 
                break
            try:
                train_df = df[df['station_id'].isin(station_split[i][j]['train'])].groupby('Time').get_group(time)
                test_df = df[df['station_id'].isin(station_split[i][j]['test'])].groupby('Time').get_group(time)
                X_train = train_df[["longitude", "latitude"]]
                y_train = train_df[["PM2.5"]]
                X_test = test_df[["longitude", "latitude"]]
                y_test = test_df[["PM2.5"]]
                krig = OrdinaryKriging(np.array(X_train)[:,0], np.array(X_train)[:, 1], y_train.values)
                prediction, var = krig.execute("points", np.array(X_test)[:, 0], np.array(X_test)[:, 1])
                error = np.sqrt( mean_squared_error( prediction, y_test ))
                errors2[hello] = error
            except Exception as e:
                print(e)
                continue
        break
    break

zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
0.0159835597670967
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array to reduction operation maximum which has no identity
zero-size array

In [36]:
a = np.mean([errors[k] for k in errors2.keys() if errors.get(k, None) is not None])
b = np.mean([errors2[k] for k in errors2.keys() if errors.get(k, None) is not None])

a, b

(16.35166485916833, 15.660106700110022)

In [37]:
print("XGB Errors (50, 1, 50)", np.mean(errors))

TypeError: unsupported operand type(s) for /: 'dict' and 'int'

In [ ]:
print("XGB Errors Default", 25.016934572865324)

In [ ]:
df

In [ ]:
df["time"].values

In [ ]:
np.mean(error)

In [ ]:
gpr.fit(train_df[["longitude", "latitude"]], train_df["PM2.5"])

In [ ]:
gpr.predict(test_df[["longitude", "latitude"]]).mean()

In [ ]:
gpr.predict(test_df[["longitude", "latitude"]]), test_df[["PM2.5"]].values

In [ ]:
gpr.predict(test_df[["longitude", "latitude"]])[0]